In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.1 MB/s eta 0:00:00


In [2]:
# Load necessary libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [3]:
import os

In [4]:
import pandas as pd
import urllib.request
import tarfile
import os

# Download IMDB dataset
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
urllib.request.urlretrieve(url, "aclImdb_v1.tar.gz")

# Extract IMDB dataset
with tarfile.open("aclImdb_v1.tar.gz", "r:gz") as tar:
    tar.extractall()

# Load IMDB dataset
imdb_df = pd.DataFrame(columns=['review', 'sentiment'])
for dataset in ['train', 'test']:
    for sentiment in ['pos', 'neg']:
        path = f'aclImdb/{dataset}/{sentiment}'
        for filename in os.listdir(path):
            with open(f'{path}/{filename}', 'r') as file:
                review = file.read()
            sentiment_value = 1 if sentiment == 'pos' else 0
            imdb_df = imdb_df.append({'review': review, 'sentiment': sentiment_value}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-4-a8f4c7381a3f>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  imdb_df = imdb_df.append({'review': review, 'sentiment': sentiment_value}, ignore_index=True)
<ipython-input-4-a8f4c7381a3f>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  imdb_df = imdb_df.append({'review': review, 'sentiment': sentiment_value}, ignore_index=True)
<ipython-input-4-a8f4c7381a3f>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  imdb_df = imdb_df.append({'review': review, 'sentiment': sentiment_value}, ignore_index=True)
<ipython-input-4-a8f4c7381a3f>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [5]:
# Count the number of instances
num_instances = len(imdb_df)

# Count the number of positive and negative instances
num_positive = imdb_df['sentiment'].value_counts()[1]
num_negative = imdb_df['sentiment'].value_counts()[0]

# Print the results
print(f'Total number of instances: {num_instances}')
print(f'Number of positive instances: {num_positive}')
print(f'Number of negative instances: {num_negative}')


Total number of instances: 50000
Number of positive instances: 25000
Number of negative instances: 25000


In [6]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00


In [7]:
# Convert sentiment values to numeric
import numpy as np
from sklearn.model_selection import train_test_split
imdb_df['sentiment'] = pd.to_numeric(imdb_df['sentiment'], errors='coerce')
imdb_df = imdb_df.dropna()

# Preprocess dataset
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-uncased', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained('xlnet-large-cased')

input_ids = []
attention_masks = []

for review in imdb_df['review']:
    encoded_dict = tokenizer.encode_plus(
                        review,                     
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(imdb_df['sentiment'].values, dtype=torch.long)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
# Split dataset into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.2)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                                 random_state=42, test_size=0.2)

In [9]:
batch_size = 16
epochs = 3

In [10]:
# Define XLNet model for sequence classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-large-cased', num_labels=2)


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [11]:
# Set the device to run the model on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 1024)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedF

In [12]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# from transformers.pipelines.base import DataLoader
# Create the DataLoader for training data
from torch.utils.data import TensorDataset,DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Create the DataLoader for validation data
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [14]:
# Define the training loop
for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    # Track the training loss and accuracy
    total_train_loss = 0
    total_train_accuracy = 0

    # Iterate over the training data
    for step, batch in enumerate(train_dataloader):
        # Clear the gradients
        model.zero_grad()

        # Move the batch to the device
        batch_inputs = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch_inputs[0],
                  'attention_mask': batch_inputs[1],
                  'labels': batch_inputs[2]}

        # Perform the forward pass
        outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]

        # Perform the backward pass and update the parameters
        loss.backward()
        optimizer.step()

        # Track the training loss and accuracy
        total_train_loss += loss.item()
        preds = torch.argmax(logits, dim=1).flatten()
        accuracy = (preds == inputs['labels']).float().mean()
        total_train_accuracy += accuracy.item()

        # Print training progress
        if step % 50 == 0:
            print(f'Epoch: {epoch + 1}, Batch: {step}, Training Loss: {total_train_loss / (step + 1)}, Training Accuracy: {total_train_accuracy / (step + 1)}')


Epoch: 1, Batch: 0, Training Loss: 0.8797512054443359, Training Accuracy: 0.4375
Epoch: 1, Batch: 50, Training Loss: 0.5831475146845275, Training Accuracy: 0.6875
Epoch: 1, Batch: 100, Training Loss: 0.4204544726353471, Training Accuracy: 0.7995049504950495
Epoch: 1, Batch: 150, Training Loss: 0.3499963763015752, Training Accuracy: 0.8414735099337748
Epoch: 1, Batch: 200, Training Loss: 0.3272164111391674, Training Accuracy: 0.8591417910447762
Epoch: 1, Batch: 250, Training Loss: 0.30853678671962237, Training Accuracy: 0.87300796812749
Epoch: 1, Batch: 300, Training Loss: 0.28900687269668446, Training Accuracy: 0.8830980066445183
Epoch: 1, Batch: 350, Training Loss: 0.2776524852271433, Training Accuracy: 0.8892450142450142
Epoch: 1, Batch: 400, Training Loss: 0.2649526174582002, Training Accuracy: 0.8946384039900249
Epoch: 1, Batch: 450, Training Loss: 0.2596841965845379, Training Accuracy: 0.8995288248337029
Epoch: 1, Batch: 500, Training Loss: 0.2573939852171137, Training Accuracy: 0

In [15]:
# Define the evaluation loop
model.eval()
total_val_loss = 0
total_val_accuracy = 0

# Iterate over the validation data
for batch in val_dataloader:
    # Move the batch to the device
    batch_inputs = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch_inputs[0],
              'attention_mask': batch_inputs[1],
              'labels': batch_inputs[2]}

    # Disable gradient calculation
    with torch.no_grad():
        # Perform the forward pass
        outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]

    # Track the validation loss and accuracy
    total_val_loss += loss.item()
    preds = torch.argmax(logits, dim=1).flatten()
    accuracy = (preds == inputs['labels']).float().mean()
    total_val_accuracy += accuracy.item()

# Calculate the average validation loss and accuracy
avg_val_loss = total_val_loss / len(val_dataloader)
avg_val_accuracy = total_val_accuracy / len(val_dataloader)
print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_accuracy}')


Validation Loss: 0.20195649402756244, Validation Accuracy: 0.9411


Here we have used seqence length we have used is 256 and batch size is 16.